In [1]:
#MOCK DATA GENERATION ----

import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Seed for reproducibility
random.seed(42)
np.random.seed(42)

# Define mock wallet IDs (from the provided Google Sheet link, picking a few examples)
mock_wallet_ids = [
    '0xfaa0768bde629806739c3a4620656c5d26f44ef2', # Example 1
    '0x732A4620656c5d26f44ef2732A4620656c5d26f4', # Example 2
    '0x1a2b3c4d5e6f7a8b9c0d1e2f3a4b5c6d7e8f9a0b', # Example 3
    '0x9f8e7d6c5b4a3f2e1d0c9b8a7f6e5d4c3b2a1f0e', # Example 4 (more risky)
    '0x112233445566778899aabbccddeeff0011223344'  # Example 5 (less active)
]

# Define possible Compound V2/V3 actions
actions = ['Supply', 'Borrow', 'RepayBorrow', 'Withdraw', 'LiquidateBorrow']
assets = ['USDC', 'DAI', 'ETH', 'WBTC'] # Common assets on Compound

# Generate mock transaction data
transactions_data = []
start_date = datetime(2023, 1, 1)

for wallet_id in mock_wallet_ids:
    num_transactions = random.randint(5, 50) # Each wallet has 5 to 50 transactions
    for _ in range(num_transactions):
        action = random.choice(actions)
        asset = random.choice(assets)
        amount = round(random.uniform(10, 5000) * (10**random.randint(6,18))) # Simulate varying token decimals
        timestamp = start_date + timedelta(days=random.randint(0, 364), hours=random.randint(0, 23))

        # Simulate some risky behavior for specific wallets
        if wallet_id == '0x9f8e7d6c5b4a3f2e1d0c9b8a7f6e5d4c3b2a1f0e' and random.random() < 0.15: # 15% chance of liquidation
            action = 'LiquidateBorrow'
            amount = round(random.uniform(1000, 10000) * (10**random.randint(6,18))) # Larger liquidation amounts

        transactions_data.append({
            'wallet_id': wallet_id,
            'action': action,
            'asset': asset,
            'amount': str(amount), # Store as string to mimic raw data
            'timestamp': int(timestamp.timestamp()), # Unix timestamp
            'tx_hash': ''.join(random.choices('0123456789abcdef', k=64)) # Mock transaction hash
        })

# Create DataFrame
df = pd.DataFrame(transactions_data)

# Convert timestamp to datetime objects for easier manipulation
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

print("Mock Transaction Data Generated:")
print(df.head())
print(f"\nTotal mock transactions: {len(df)}")
print(f"Unique wallets in mock data: {df['wallet_id'].nunique()}")


Mock Transaction Data Generated:
                                    wallet_id           action asset  \
0  0xfaa0768bde629806739c3a4620656c5d26f44ef2           Supply  USDC   
1  0xfaa0768bde629806739c3a4620656c5d26f44ef2      RepayBorrow  WBTC   
2  0xfaa0768bde629806739c3a4620656c5d26f44ef2  LiquidateBorrow   DAI   
3  0xfaa0768bde629806739c3a4620656c5d26f44ef2         Withdraw  USDC   
4  0xfaa0768bde629806739c3a4620656c5d26f44ef2      RepayBorrow  USDC   

                  amount           timestamp  \
0          3710336993802 2023-04-24 22:30:00   
1          1346026971119 2023-10-18 16:30:00   
2       6506734078538163 2023-10-08 23:30:00   
3              831539236 2023-07-19 02:30:00   
4  444966640795848867840 2023-10-10 22:30:00   

                                             tx_hash  
0  bae1603803a839c0cb52f511d9cb8f68d9d9b0341314a5...  
1  5a6e743849e63f8101ac616f8fd0ba84a167fe482ed4a9...  
2  4d638b34fa781359331a3ed13a32e97cc31667baf165d3...  
3  ec4bbca7330b0507e808d1

In [2]:
#DATA PREPARATION ----

df['amount_numeric'] = pd.to_numeric(df['amount'], errors='coerce').fillna(0)

# Group by wallet_id to create aggregated features
wallet_features = df.groupby('wallet_id').agg(
    total_transactions=('tx_hash', 'count'),
    first_transaction_date=('timestamp', 'min'),
    last_transaction_date=('timestamp', 'max'),

    # Counts of specific actions
    supply_count=('action', lambda x: (x == 'Supply').sum()),
    borrow_count=('action', lambda x: (x == 'Borrow').sum()),
    repay_count=('action', lambda x: (x == 'RepayBorrow').sum()),
    withdraw_count=('action', lambda x: (x == 'Withdraw').sum()),
    liquidate_count=('action', lambda x: (x == 'LiquidateBorrow').sum()),

    # Total amounts for specific actions (in raw token units)
    total_supplied_amount=('amount_numeric', lambda x: x[df.loc[x.index, 'action'] == 'Supply'].sum()),
    total_borrowed_amount=('amount_numeric', lambda x: x[df.loc[x.index, 'action'] == 'Borrow'].sum()),
    total_repaid_amount=('amount_numeric', lambda x: x[df.loc[x.index, 'action'] == 'RepayBorrow'].sum()),
    total_withdrawn_amount=('amount_numeric', lambda x: x[df.loc[x.index, 'action'] == 'Withdraw'].sum()),
    total_liquidated_amount=('amount_numeric', lambda x: x[df.loc[x.index, 'action'] == 'LiquidateBorrow'].sum()),

    # Number of unique assets interacted with
    unique_assets_count=('asset', 'nunique')

).reset_index()

# Calculate derived features for risk assessment

# Account Age: How long the wallet has been active
wallet_features['account_age_days'] = (wallet_features['last_transaction_date'] - wallet_features['first_transaction_date']).dt.days
wallet_features['account_age_days'] = wallet_features['account_age_days'].fillna(0) # For wallets with single transaction

# Borrow-to-Supply Ratio: Higher ratio might indicate higher leverage/risk
# Add epsilon to avoid division by zero
wallet_features['borrow_to_supply_ratio'] = wallet_features['total_borrowed_amount'] / (wallet_features['total_supplied_amount'] + 1e-9)
wallet_features['borrow_to_supply_ratio'] = wallet_features['borrow_to_supply_ratio'].replace([np.inf, -np.inf], 0).fillna(0)

# Repay-to-Borrow Ratio: Higher ratio indicates good repayment behavior
wallet_features['repay_to_borrow_ratio'] = wallet_features['total_repaid_amount'] / (wallet_features['total_borrowed_amount'] + 1e-9)
wallet_features['repay_to_borrow_ratio'] = wallet_features['repay_to_borrow_ratio'].replace([np.inf, -np.inf], 0).fillna(0)

# Net Borrowed Amount: Outstanding debt
wallet_features['net_borrowed_amount'] = wallet_features['total_borrowed_amount'] - wallet_features['total_repaid_amount']

# Liquidation Frequency: How often the wallet was liquidated
wallet_features['liquidation_frequency'] = wallet_features['liquidate_count'] / (wallet_features['total_transactions'] + 1e-9)
wallet_features['liquidation_frequency'] = wallet_features['liquidation_frequency'].fillna(0)

# Select final features for the scoring model
features_for_scoring = [
    'total_transactions', 'supply_count', 'borrow_count', 'repay_count',
    'withdraw_count', 'liquidate_count', 'total_supplied_amount',
    'total_borrowed_amount', 'total_repaid_amount', 'total_withdrawn_amount',
    'total_liquidated_amount', 'unique_assets_count', 'account_age_days',
    'borrow_to_supply_ratio', 'repay_to_borrow_ratio', 'net_borrowed_amount',
    'liquidation_frequency'
]

# Ensure all selected features are numeric and handle potential NaNs from aggregation
for col in features_for_scoring:
    wallet_features[col] = pd.to_numeric(wallet_features[col], errors='coerce').fillna(0)

print("\nEngineered Features for Risk Scoring:")
print(wallet_features[features_for_scoring].head())
print(f"\nNumber of wallets with engineered features: {len(wallet_features)}")


Engineered Features for Risk Scoring:
   total_transactions  supply_count  borrow_count  repay_count  \
0                  39             8             9            7   
1                  25             2             7            7   
2                  18             1             4            4   
3                  13             3             2            0   
4                  45            12             8            6   

   withdraw_count  liquidate_count  total_supplied_amount  \
0              10                5           5.092530e+20   
1               7                2           4.000005e+15   
2               3                6           3.348003e+16   
3               5                3           4.660640e+16   
4               9               10           3.751955e+18   

   total_borrowed_amount  total_repaid_amount  total_withdrawn_amount  \
0           4.206731e+21         1.482454e+18            9.151445e+21   
1           4.880502e+21         6.823333e+20      

In [3]:
#RISK SCORING MODEL ----

def calculate_risk_score(wallet_features_df):
    """
    Calculates a risk score (0-1000) for each wallet based on engineered features.
    Higher scores indicate lower risk (more reliable usage).

    Args:
        wallet_features_df (pd.DataFrame): DataFrame containing engineered features for wallets.

    Returns:
        pd.DataFrame: DataFrame with 'wallet_id' and 'score' columns.
    """
    print("\nCalculating Risk Scores (0-1000)...")

    # Initialize scores. Start with a base score, then add/subtract based on behavior.
    scores = pd.Series(np.full(len(wallet_features_df), 500), index=wallet_features_df.index)

    # --- Positive Indicators (Lower Risk, Higher Score) ---

    # 1. Repay-to-Borrow Ratio: Wallets that repay more than they borrow are less risky.
    # Max 200 points for excellent repayment. Using log1p to handle large ratios gracefully.
    scores += (np.log1p(wallet_features_df['repay_to_borrow_ratio']) * 40).clip(0, 200)

    # 2. Total Supplied Amount: Wallets that supply more capital are generally more committed and less risky.
    # Normalize by max supplied amount to give points. Max 100 points.
    if wallet_features_df['total_supplied_amount'].max() > 0:
        scores += (wallet_features_df['total_supplied_amount'] / wallet_features_df['total_supplied_amount'].max() * 100).fillna(0).clip(0, 100)

    # 3. Account Age: Older, consistently active accounts might be more reliable.
    # Normalize by max account age. Max 50 points.
    if wallet_features_df['account_age_days'].max() > 0:
        scores += (wallet_features_df['account_age_days'] / wallet_features_df['account_age_days'].max() * 50).fillna(0).clip(0, 50)

    # 4. Activity Level: More transactions can indicate active, non-dormant usage.
    # Normalize by max total transactions. Max 30 points.
    if wallet_features_df['total_transactions'].max() > 0:
        scores += (wallet_features_df['total_transactions'] / wallet_features_df['total_transactions'].max() * 30).fillna(0).clip(0, 30)


    # --- Negative Indicators (Higher Risk, Lower Score) ---

    # 1. Liquidation Count: Wallets that have been liquidated are high risk.
    # Each liquidation significantly reduces the score. Max reduction 400 points.
    scores -= (wallet_features_df['liquidate_count'] * 100).clip(0, 400)

    # 2. Liquidation Frequency: High frequency indicates consistent risky behavior.
    # Max reduction 150 points.
    scores -= (wallet_features_df['liquidation_frequency'] * 150).clip(0, 150)

    # 3. Net Borrowed Amount: Significant outstanding debt indicates higher risk.
    # Normalize by max net borrowed amount. Max reduction 200 points.
    if wallet_features_df['net_borrowed_amount'].max() > 0:
        scores -= (wallet_features_df['net_borrowed_amount'] / wallet_features_df['net_borrowed_amount'].max() * 200).fillna(0).clip(0, 200)

    # 4. Borrow-to-Supply Ratio: Very high ratio implies over-leveraging.
    # Max reduction 100 points. Using log1p to dampen extreme values.
    scores -= (np.log1p(wallet_features_df['borrow_to_supply_ratio']) * 20).clip(0, 100)


    # Final clamping of scores to ensure they are within the 0-1000 range
    final_scores = scores.clip(0, 1000).round(0).astype(int)

    # Create the output DataFrame
    risk_scores_df = pd.DataFrame({
        'wallet_id': wallet_features_df['wallet_id'],
        'score': final_scores
    })

    print("Risk scores calculated.")
    print("First 10 wallet scores:")
    print(risk_scores_df.head(10))
    print(f"\nScore distribution (min, max, mean): {risk_scores_df['score'].min()}, {risk_scores_df['score'].max()}, {risk_scores_df['score'].mean():.2f}")

    return risk_scores_df

# --- Execute Risk Score Calculation ---
final_risk_scores = calculate_risk_score(wallet_features.copy()) # Use a copy to avoid modifying original features_df



Calculating Risk Scores (0-1000)...
Risk scores calculated.
First 10 wallet scores:
                                    wallet_id  score
0  0x112233445566778899aabbccddeeff0011223344     11
1  0x1a2b3c4d5e6f7a8b9c0d1e2f3a4b5c6d7e8f9a0b     60
2  0x732A4620656c5d26f44ef2732A4620656c5d26f4    268
3  0x9f8e7d6c5b4a3f2e1d0c9b8a7f6e5d4c3b2a1f0e    102
4  0xfaa0768bde629806739c3a4620656c5d26f44ef2    130

Score distribution (min, max, mean): 11, 268, 114.20


In [4]:
# CSV FILE ----

output_csv_filename = 'https://docs.google.com/spreadsheets/d/1ZzaeMgNYnxvriYYpe8PE7uMEblTI0GV5GIVUnsP-sBs/edit?gid=0#gid=0'
final_risk_scores.to_csv(output_csv_filename, index=False)

print(f"\nWallet risk scores saved to '{output_csv_filename}'.")
print("CSV file content (first 5 rows):")
print(pd.read_csv(output_csv_filename).head())


Wallet risk scores saved to 'https://docs.google.com/spreadsheets/d/1ZzaeMgNYnxvriYYpe8PE7uMEblTI0GV5GIVUnsP-sBs/edit?gid=0#gid=0'.
CSV file content (first 5 rows):
  <!DOCTYPE html><html lang="en-GB"><head><script nonce="oPGw9GItihAXGydL1XcTOQ">var DOCS_timing={}; DOCS_timing['pls']=new Date().getTime();</script><meta property="og:title" content="Wallet id"><meta property="og:type" content="article"><meta property="og:site_name" content="Google Docs"><meta property="og:url" content="https://docs.google.com/spreadsheets/d/1ZzaeMgNYnxvriYYpe8PE7uMEblTI0GV5GIVUnsP-sBs/edit?gid=0&amp;usp=embed_facebook"><meta property="og:image" content="https://lh7-us.googleusercontent.com/docs/AHkbwyLie8SHcbbY59yLzABwkh1YrGDN_DOKgmmBlZ0mfqa6jBkUyymS-FadD0__1XMocbfZtr_YLcO_LIQQIhcl-P9jtgPCt7uE5CDFd7dpDiY_yVsZfww=w1200-h630-p"><meta property="og:image:width" content="1200"><meta property="og:image:height" content="630"><meta name="google" content="notranslate"><meta http-equiv="X-UA-Compatible" content="